In [50]:
using Pkg

In [51]:
Pkg.add("Plots")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [52]:
using DelimitedFiles
using Plots

In [53]:
## Read in training data (format is in rows of 217 numbers, where the first number is the label 0-9, and the remaining 256 numbers indicate the pixel intensity)
training_data = readdlm("zip.train")
training_data_y = convert(Vector{Int8}, training_data[:,1])
training_data[:,1] .= 1 #create column of ones for intercept
training_data

7291×257 Matrix{Float64}:
 1.0  -1.0    -1.0    -1.0    -1.0    -1.0    …  -1.0    -1.0    -1.0    -1.0
 1.0  -1.0    -1.0    -1.0    -0.813  -0.671     -0.095  -0.671  -0.828  -1.0
 1.0  -1.0    -1.0    -1.0    -1.0    -1.0       -1.0    -1.0    -1.0    -1.0
 1.0  -1.0    -1.0    -1.0    -1.0    -1.0       -1.0    -1.0    -1.0    -1.0
 1.0  -1.0    -1.0    -1.0    -1.0    -1.0       -0.199  -0.883  -1.0    -1.0
 1.0  -1.0    -1.0    -1.0    -1.0    -1.0    …  -1.0    -1.0    -1.0    -1.0
 1.0  -1.0    -1.0    -1.0    -0.83    0.442     -1.0    -1.0    -1.0    -1.0
 1.0  -1.0    -1.0    -1.0    -1.0    -1.0       -1.0    -1.0    -1.0    -1.0
 1.0  -1.0    -1.0    -1.0    -1.0    -1.0       -1.0    -1.0    -1.0    -1.0
 1.0  -1.0    -1.0    -1.0    -1.0    -1.0       -1.0    -1.0    -1.0    -1.0
 1.0  -1.0    -1.0    -1.0    -1.0    -1.0    …  -1.0    -1.0    -1.0    -1.0
 1.0  -1.0    -1.0    -1.0    -1.0    -1.0       -1.0    -1.0    -1.0    -1.0
 1.0  -1.0    -1.0    -1.0    -1.0    

In [57]:
## Filter for '2' and '3'. 
A = training_data[2 .<= training_data_y .<= 3,:]
b = training_data_y[2 .<= training_data_y .<= 3]

## compute least squares coefficients
x = A\b

## compute training data
train_predictions = round.(Int8, training_data * x)
lr_train_error = sum(train_predictions .== training_data_y)/length(training_data_y)

0.19009738033191606

In [56]:
test_data = readdlm("zip.test")
test_data_y = convert(Vector{Int8}, test_data[:,1])
test_data[:,1] .= 1

test_data = test_data[2 .<= test_data_y .<= 3,:]
test_data_y = test_data_y[2 .<= test_data_y .<= 3]

test_predictions = round.(Int8, test_data * x)
lr_test_error = sum(test_predictions .== test_data_y)/length(test_data_y)

0.9587912087912088